In [1]:
# %matplotlib inline
# import matplotlib.pyplot as plt
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVC

In [23]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

data_file_list = ['csv4verIQ_nanNetwork', 'csv4verIQ_maleNetwork', 'csv4verIQ_genderNetwork', 
                  'csv4verIQ_femaleNetwork','csv4perIQ_nanNetwork','csv4perIQ_maleNetwork',
                  'csv4perIQ_genderNetwork', 'csv4perIQ_femaleNetwork']
data_path = "../data/data4SVR_IQs/"
for data_file in data_file_list:
    print('###############################################\n')
    print('Data:{}\n'.format(data_file))
    # comma delimited is the default
    df = pd.read_csv(os.path.join(data_path, data_file + '.csv'), header = 0)

    # put the original column names in a python list
    original_headers = list(df.columns.values)
    feature_name_list = original_headers[1:]
    # remove the non-numeric columns
    df = df._get_numeric_data()

    # put the numeric column names in a python list
    numeric_headers = list(df.columns.values)

    # create a numpy array with the numeric values for input into scikit-learn
    numpy_array = np.array(df.as_matrix())
    Y = numpy_array[:,0]
    X = numpy_array[:,1:]
    print 'X:', X.shape, 'Y:', Y.shape
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    
    Y_both = []
    loo = LeaveOneOut()
    loo.get_n_splits(X)
    C_range = [2**each for each in range(-2,3)]
    print('C_range:{}'.format(C_range))
    best_C_val_found = False
    for train_index, test_index in loo.split(X):
#         print(str(test_index))
        
        '''
        mean_score_list = []
        for C_val in C_range:
            scores = cross_val_score(
                (make_pipeline(StandardScaler(), SVC(C=C_val))), 
                X[train_index,:], Y[train_index], 
#                 scoring="neg_mean_squared_error",
                cv=5)
            mean_score_list.append(np.mean(scores))
        min_score_idx = np.argmin(mean_score_list)
        max_score_idx = np.argmax(mean_score_list)
        print(mean_score_list)
        print('Max_score_idx:{}, value:{}, C:{}'.format(
            max_score_idx, mean_score_list[max_score_idx], C_range[max_score_idx]))
#         print('min_score_idx:{}, value:{}, C:{}'.format(
#             min_score_idx, mean_score_list[min_score_idx], C_range[min_score_idx]))
        '''
        if not best_C_val_found:
            print('Looking for best C')
            
            mean_score_list = []
            for C_val in C_range:
                scores = cross_val_score(
                    (make_pipeline(StandardScaler(), SVC(C=C_val))), 
                    X[train_index,:], Y[train_index], 
                    scoring="neg_mean_squared_error",
                    cv=5)
                mean_score_list.append(np.mean(scores))
            min_score_idx = np.argmin(mean_score_list)
            max_score_idx = np.argmax(mean_score_list)
            print(mean_score_list)
            best_C_val = C_range[max_score_idx]
            print('Max_score_idx:{}, value:{}, Best_C:{}'.format(
                max_score_idx, mean_score_list[max_score_idx], best_C_val))
#             print('min_score_idx:{}, value:{}, C:{}'.format(
#                 min_score_idx, mean_score_list[min_score_idx], C_range[min_score_idx]))
            best_C_val_found = True
    
        clf = SVC(C=best_C_val)
        clf.fit(X[train_index,:], Y[train_index])
        Y_both.append([Y[test_index][0], clf.predict(X[test_index,:])[0]])
    import csv
    with open(os.path.join(data_path, data_file+'_pred.csv'), 'wb') as myfile:
        myfile.write('True_Y, Predicted_Y\n')
        for each in Y_both:
            myfile.write('{},{}\n'.format(each[0], each[1]))
        '''
        wr = csv.writer(myfile)
        wr.writerow(["True Y", "Predicted Y"])
        wr.writerow(Y_both)
        '''

###############################################

Data:csv4verIQ_nanNetwork

X: (394, 36) Y: (394,)
C_range:[0.25, 0.5, 1, 2, 4]
Looking for best C
[-93.650018906668947, -95.391803599226847, -102.36940152561418, -110.73790173853538, -126.38599616082119]
Max_score_idx:0, value:-93.6500189067, Best_C:0.25
###############################################

Data:csv4verIQ_maleNetwork

X: (394, 4) Y: (394,)
C_range:[0.25, 0.5, 1, 2, 4]
Looking for best C
[-93.650018906668947, -93.690525235782872, -97.299734763136257, -118.17411391072531, -129.46151901617691]
Max_score_idx:0, value:-93.6500189067, Best_C:0.25
###############################################

Data:csv4verIQ_genderNetwork

X: (394, 11) Y: (394,)
C_range:[0.25, 0.5, 1, 2, 4]
Looking for best C
[-93.650018906668947, -93.906919533260648, -98.284695931612518, -101.28792458687892, -101.73446949678204]
Max_score_idx:0, value:-93.6500189067, Best_C:0.25
###############################################

Data:csv4verIQ_femaleNetwork

X: (39